# Bayesian Linear Regression

The purpose of this notebook is for testing and exploring features of the Bayesian Linear Regression

In [14]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as ss

import plotly
import plotly.express as px

import pymc3 as pm
from pymc3 import  *

# Generate test Data using the base parameters and attributes

In [15]:
base_params = dict(
    alpha = 50,
    beta = 3,
    gamma=0,
)

noise = 0.2

attributes = dict(
    material = dict(
        timber = dict(
            alpha = -10,
            beta = -0.5,
        ),
        concrete = dict(
            alpha = 0,
            beta = 0,
        ),
        composite = dict(
            alpha = 30,
            beta = 2,
        ),
    ),
    treatment = dict(
        bad = dict(
            alpha = -5,
            beta = 0,
        ),
        good = dict(
            alpha = 0,
            beta = 0,
        ),
        great = dict(
            alpha = 5,
            beta = 0,
        ),
    ),
)

In [16]:
size = 1000
df = pd.DataFrame()

for att, trait in attributes.items():

    # Get a random mix of attributes
    df[att] = np.random.choice(list(trait), size=1000) 

    # Get the parameters for each attribute
    df_params = df[att].map(trait).apply(pd.Series).add_suffix("_" + att)

    # Append DataFrames
    df = pd.concat([df, df_params], axis=1)

#Combine the parameters together
for param, base_value in base_params.items():
    param_cols = [col for col in df if col.startswith(param)]
    df[param] = base_value + df[param_cols].sum(axis=1)

# Add a time column
df['time'] = np.random.randint(0, 100, size=size)

# Generate a failure column
df['failure'] = ss.weibull_min.cdf(df['time'], df['beta'], scale=df['alpha'], loc=df['gamma'])

# Add some noise
df['failure'] = df['failure'] * np.random.uniform(1 - noise, 1 + noise, size=size)

# Generate a condition column
df['condition'] = 1 - (df['failure'] * 2)
df['condition'] = np.where(df['condition'] < 0, np.NaN, df['condition'])



In [17]:
px.scatter(df, x="time", y="condition", color="material", title='Maintenance Strategy Costs')

In [18]:
# Get the numerical and categorical attributes
att = dict(
    categorical = list(attributes),
    numerical = [],
)

x_var = list(pd.get_dummies(df[att['categorical'] + att['numerical']], columns = att['categorical'], prefix_sep="_"))
y_var = 'failure'

cat_var = list(attributes)

X = pd.get_dummies(df, columns = att['categorical'], prefix_sep="_")

## Formula
Get the formula for the condition

In [19]:
def get_formula(y_var, x_var):
    
    formula = "{} ~ {}".format(y_var,' + '.join(x_var))
    
    return formula

In [20]:
formula = get_formula(y_var, x_var)
formula

'failure ~ material_composite + material_concrete + material_timber + treatment_bad + treatment_good + treatment_great'

## Establish the priors

In [21]:
def get_priors_simple(cat_var, perfect, degraded):
    """
    Use regression results to establish priors. The
    """
    priors = dict()
    priors["Intercept"] = pm.Uniform.dist(lower=perfect, upper=degraded)
    
    for at in cat_var:
        priors[at] = pm.Normal.dist(mu=0, sd = (degraded-perfect)/6)

    return priors

In [22]:
def get_priors_informed(cat_var, perfect, degraded, mean, sd):
    """
    Use regression results to establish priors. The
    """
    priors = dict()
    #priors["Intercept"] = pm.Uniform.dist(lower=perfect, upper=degraded)
    priors["Intercept"] = pm.Normal.dist(mu=mean, sd=sd)
    
    for at in cat_var:
        priors[at] = pm.Normal.dist(mu=0, sd = (degraded-perfect)/6)
    
    return priors

In [23]:
def get_priors(cat_var, lower, upper, mean, sd):
    try:
        priors = get_priors_informed(cat_var, lower, upper, mean, sd)
    except:
        priors = get_priors_simple(cat_var,lower, upper)
    return priors

In [24]:
priors = get_priors(cat_var=cat_var, lower=0, upper=1, mean=df[y_var].mean(), sd=df[y_var].std())
priors

{'Intercept': <pymc3.distributions.continuous.Normal at 0x10bb0b51250>,
 'material': <pymc3.distributions.continuous.Normal at 0x10bb0c85190>,
 'treatment': <pymc3.distributions.continuous.Normal at 0x10bb0b3a460>}

## Get the model trace

In [25]:
def get_model_trace(data, formula, priors):
    """
    Pass a dataframe, a formula and some priors to get 
    """
    
    with pm.Model() as model:

        # The prior for the model parameters will be a normal distribution
        family = pm.glm.families.Normal()

        # Creating the model requires a formula and data (and optionally a family)
        pm.GLM.from_formula(formula, data = data, family=family, priors=priors)
        
        # Perform Markov Chain Monte Carlo sampling
        trace = pm.sample(init='adapt_diag', draws=2000, tune=1000, cores=2, chains=2, progressbar=True)
        
    return trace

In [26]:
trace = get_model_trace(X, formula, priors)


Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sd, treatment_great, treatment_good, treatment_bad, material_timber, material_concrete, material_composite, Intercept]


ValueError: Not enough samples to build a trace.

In [13]:
pm.plot_posterior(trace)

NameError: name 'trace' is not defined

In [ ]:
with pm.Model() as model_2:

    

